In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
from pylab import figure, axes, pie, title, savefig
import time
import csv
import sys
import plotly.express as px

In [2]:
df = pd.read_csv("/Users/hamin/bithumb_bot_data/include/up-btc/upbit-2019-05-btc-krw.csv")

In [3]:
df.dtypes

timestamp     object
quantity     float64
price          int64
fee          float64
amount         int64
side           int64
dtype: object

In [4]:
## converting timestamp column type 'object' to 'datetime'
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df.dtypes

timestamp    datetime64[ns]
quantity            float64
price                 int64
fee                 float64
amount                int64
side                  int64
dtype: object

In [5]:
## converting timestamp to detailed date column
df['timestamp_year_month_day']       = df['timestamp'].dt.date         # YYYY-MM-DD(문자)
df['timestamp_year']       = df['timestamp'].dt.year         # 연(4자리숫자)
df['timestamp_month']      = df['timestamp'].dt.month        # 월(숫자)
df['timestamp_month_name'] = df['timestamp'].dt.month_name() # 월(문자)

df['timestamp_day']        = df['timestamp'].dt.day          # 일(숫자)
df['timestamp_time']       = df['timestamp'].dt.time         # HH:MM:SS(문자)
df['timestamp_hour']       = df['timestamp'].dt.hour         # 시(숫자)
df['timestamp_minute']     = df['timestamp'].dt.minute       # 분(숫자)
df['timestamp_second']     = df['timestamp'].dt.second       # 초(숫자)

In [6]:
df

,timestamp,quantity,price,fee,amount,side,timestamp_year_month_day,timestamp_year,timestamp_month,timestamp_month_name,timestamp_day,timestamp_time,timestamp_hour,timestamp_minute,timestamp_second
0,2019-05-16 13:32:00,0.022435,9568000,107.32,-214764,0,2019-05-16,2019,5,May,16,13:32:00,13,32,0
1,2019-05-16 13:32:00,0.004065,9568000,19.44,-38916,0,2019-05-16,2019,5,May,16,13:32:00,13,32,0
2,2019-05-16 13:32:00,0.007700,9568000,36.83,-73711,0,2019-05-16,2019,5,May,16,13:32:00,13,32,0
3,2019-05-16 13:32:00,0.800815,9568000,3831.09,-7666025,0,2019-05-16,2019,5,May,16,13:32:00,13,32,0
4,2019-05-16 13:32:00,0.052347,9568000,250.42,-501105,0,2019-05-16,2019,5,May,16,13:32:00,13,32,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17617,2019-05-31 23:38:00,0.000484,10221000,2.47,4943,1,2019-05-31,2019,5,May,31,23:38:00,23,38,0
17618,2019-05-31 23:38:00,0.009779,10221000,49.97,99899,1,2019-05-31,2019,5,May,31,23:38:00,23,38,0
17619,2019-05-31 23:38:00,0.028431,10221000,145.29,290446,1,2019-05-31,2019,5,May,31,23:38:00,23,38,0
17620,2019-05-31 23:38:00,0.020316,10221000,103.82,207550,1,2019-05-31,2019,5,May,31,23:38:00,23,38,0


In [7]:
## creating 'day - amount' connection
group_day_amount = df['amount'].groupby(df['timestamp_year_month_day'])

In [8]:
group_day_amount.size()

timestamp_year_month_day
2019-05-16    1677
2019-05-17    2954
2019-05-18    1720
2019-05-19     924
2019-05-20    1249
2019-05-21    1065
2019-05-22     712
2019-05-23     622
2019-05-24     327
2019-05-25     546
2019-05-26     538
2019-05-27    1660
2019-05-28     781
2019-05-29     734
2019-05-30     576
2019-05-31    1537
Name: amount, dtype: int64

In [9]:
group_day_amount

In [10]:
## 일별 총 손익량
group_day_amount.sum()

timestamp_year_month_day
2019-05-16   -28298696
2019-05-17     8980945
2019-05-18     4329182
2019-05-19    11786617
2019-05-20     1701085
2019-05-21      178552
2019-05-22    -2784014
2019-05-23     2561143
2019-05-24    -1431054
2019-05-25     1992816
2019-05-26   -28786577
2019-05-27    21574720
2019-05-28    -5882058
2019-05-29    15174996
2019-05-30    -6327666
2019-05-31     6958409
Name: amount, dtype: int64

In [11]:
## creating 'day - earning amount' connection
df2 = df.copy()
df2 =df2[df2['amount'] > 0]
group_day_amount_positive = df2['amount'].groupby(df2['timestamp_year_month_day'])


In [12]:
# 일별 수익 transaction 수
group_day_amount_positive.size()

timestamp_year_month_day
2019-05-16     882
2019-05-17    1795
2019-05-18     963
2019-05-19     536
2019-05-20     776
2019-05-21     633
2019-05-22     384
2019-05-23     326
2019-05-24     223
2019-05-25     351
2019-05-26     349
2019-05-27     907
2019-05-28     445
2019-05-29     477
2019-05-30     344
2019-05-31     940
Name: amount, dtype: int64

In [13]:
# 일별 수익량
group_day_amount_positive.sum()

timestamp_year_month_day
2019-05-16    1463599408
2019-05-17    2664676309
2019-05-18    1747665433
2019-05-19    1048883668
2019-05-20    1439256809
2019-05-21     968504619
2019-05-22     539605210
2019-05-23     541648168
2019-05-24     320160363
2019-05-25     471263057
2019-05-26     307292576
2019-05-27    1709257916
2019-05-28     650819360
2019-05-29     605257538
2019-05-30     610111143
2019-05-31    1328782115
Name: amount, dtype: int64

In [14]:
## creating 'day - loss amount' connection
df3 = df.copy()
df3 = df3[df3['amount'] < 0]
group_day_amount_negative = df3['amount'].groupby(df3['timestamp_year_month_day'])

In [15]:
# 일별 손해 transaction 수
group_day_amount_negative.size()

timestamp_year_month_day
2019-05-16     795
2019-05-17    1156
2019-05-18     755
2019-05-19     387
2019-05-20     469
2019-05-21     431
2019-05-22     328
2019-05-23     296
2019-05-24     104
2019-05-25     194
2019-05-26     187
2019-05-27     753
2019-05-28     336
2019-05-29     256
2019-05-30     232
2019-05-31     597
Name: amount, dtype: int64

In [16]:
# 일별 손해량
group_day_amount_negative.sum()

timestamp_year_month_day
2019-05-16   -1491898104
2019-05-17   -2655695364
2019-05-18   -1743336251
2019-05-19   -1037097051
2019-05-20   -1437555724
2019-05-21    -968326067
2019-05-22    -542389224
2019-05-23    -539087025
2019-05-24    -321591417
2019-05-25    -469270241
2019-05-26    -336079153
2019-05-27   -1687683196
2019-05-28    -656701418
2019-05-29    -590082542
2019-05-30    -616438809
2019-05-31   -1321823706
Name: amount, dtype: int64

In [17]:
# 일별 절대값 총합
absolute_negative_sum=group_day_amount_negative.sum()
absolute_negative_sum = absolute_negative_sum * (-1)
absolute_total_amout=absolute_negative_sum+group_day_amount_positive.sum()
absolute_total_amout

timestamp_year_month_day
2019-05-16    2955497512
2019-05-17    5320371673
2019-05-18    3491001684
2019-05-19    2085980719
2019-05-20    2876812533
2019-05-21    1936830686
2019-05-22    1081994434
2019-05-23    1080735193
2019-05-24     641751780
2019-05-25     940533298
2019-05-26     643371729
2019-05-27    3396941112
2019-05-28    1307520778
2019-05-29    1195340080
2019-05-30    1226549952
2019-05-31    2650605821
Name: amount, dtype: int64

In [18]:
## 일별 계산 table 생성
dfs = [group_day_amount.size(), group_day_amount.sum(), group_day_amount_positive.size(), group_day_amount_positive.sum(), group_day_amount_negative.size(), group_day_amount_negative.sum(), absolute_total_amout, absolute_negative_sum]
dfs

[timestamp_year_month_day
 2019-05-16    1677
 2019-05-17    2954
 2019-05-18    1720
 2019-05-19     924
 2019-05-20    1249
 2019-05-21    1065
 2019-05-22     712
 2019-05-23     622
 2019-05-24     327
 2019-05-25     546
 2019-05-26     538
 2019-05-27    1660
 2019-05-28     781
 2019-05-29     734
 2019-05-30     576
 2019-05-31    1537
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2019-05-16   -28298696
 2019-05-17     8980945
 2019-05-18     4329182
 2019-05-19    11786617
 2019-05-20     1701085
 2019-05-21      178552
 2019-05-22    -2784014
 2019-05-23     2561143
 2019-05-24    -1431054
 2019-05-25     1992816
 2019-05-26   -28786577
 2019-05-27    21574720
 2019-05-28    -5882058
 2019-05-29    15174996
 2019-05-30    -6327666
 2019-05-31     6958409
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2019-05-16     882
 2019-05-17    1795
 2019-05-18     963
 2019-05-19     536
 2019-05-20     776
 2019-05-21     633
 2019-05-22     384
 2019-05-23     326


In [19]:
day_df_final = reduce(lambda left,right: pd.merge(left,right,on='timestamp_year_month_day'), dfs)
day_df_final.columns = [ "daily transaction", "daily income", "daily Yield transaction", "daily Yield", "daily loss transaction", "daily loss", "daily absolute amount", "daily absolute negative amount"]
day_df_final

,daily transaction,daily income,daily Yield transaction,daily Yield,daily loss transaction,daily loss,daily absolute amount,daily absolute negative amount
timestamp_year_month_day,,,,,,,,
2019-05-16,1677,-28298696,882,1463599408,795,-1491898104,2955497512,1491898104
2019-05-17,2954,8980945,1795,2664676309,1156,-2655695364,5320371673,2655695364
2019-05-18,1720,4329182,963,1747665433,755,-1743336251,3491001684,1743336251
2019-05-19,924,11786617,536,1048883668,387,-1037097051,2085980719,1037097051
2019-05-20,1249,1701085,776,1439256809,469,-1437555724,2876812533,1437555724
2019-05-21,1065,178552,633,968504619,431,-968326067,1936830686,968326067
2019-05-22,712,-2784014,384,539605210,328,-542389224,1081994434,542389224
2019-05-23,622,2561143,326,541648168,296,-539087025,1080735193,539087025
2019-05-24,327,-1431054,223,320160363,104,-321591417,641751780,321591417


In [20]:
print(day_df_final.asfreq('D'))

                          daily transaction  daily income  \
timestamp_year_month_day                                    
2019-05-16                             1677     -28298696   
2019-05-17                             2954       8980945   
2019-05-18                             1720       4329182   
2019-05-19                              924      11786617   
2019-05-20                             1249       1701085   
2019-05-21                             1065        178552   
2019-05-22                              712      -2784014   
2019-05-23                              622       2561143   
2019-05-24                              327      -1431054   
2019-05-25                              546       1992816   
2019-05-26                              538     -28786577   
2019-05-27                             1660      21574720   
2019-05-28                              781      -5882058   
2019-05-29                              734      15174996   
2019-05-30              

In [21]:
day_df_final.to_csv("/Users/hamin/bithumb_bot_data/upbit-2019-05-daily_result_table.csv", header=True, index=True, encoding = 'utf-8')


In [22]:
# 뒤집기.
day_df_final[::-1].to_csv("/Users/hamin/bithumb_bot_data/upbit-2019-05-daily_result_table.csv", header=True, index=True, encoding = 'utf-8')

In [23]:
day_df_final

,daily transaction,daily income,daily Yield transaction,daily Yield,daily loss transaction,daily loss,daily absolute amount,daily absolute negative amount
timestamp_year_month_day,,,,,,,,
2018-01-19,40,1052348,21,1334332,19,-281984,1616316.0,281984
2018-01-20,153,426,76,1239191,77,-1238765,2477956.0,1238765
2018-01-21,154,-1741,76,1201705,78,-1203446,2405151.0,1203446
2018-01-22,69,-14922,34,480676,35,-495598,976274.0,495598
2018-01-23,96,899,48,633072,48,-632173,1265245.0,632173
2018-01-24,86,-1680,42,565300,44,-566980,1132280.0,566980
2018-01-25,217,11247,109,1442316,108,-1431069,2873385.0,1431069
2018-01-31,4,-86,2,22176,2,-22262,44438.0,22262
